<a href="https://colab.research.google.com/github/souravs17031999/private-ai/blob/master/Encrypted_deep_learning_final_toy_dataset_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project - XVI
## OBJECTIVE : TO IMPLEMENT ENCRYPTED DEEP LEARNING USING PYSYFT

In [9]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0719 12:16:06.850576 139801329891200 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0719 12:16:06.871351 139801329891200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

FIRSTLY , LET'S TRAIN THE MODEL WITHOUT ENCRYPTION

In [0]:
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()
        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        


In [14]:
train()

tensor(0.7791)
tensor(0.7639)
tensor(0.7556)
tensor(0.7446)
tensor(0.7310)
tensor(0.7193)
tensor(0.7195)
tensor(0.7090)
tensor(0.6996)
tensor(0.6936)
tensor(0.6865)
tensor(0.6896)
tensor(0.6830)
tensor(0.6784)
tensor(0.6756)
tensor(0.6745)
tensor(0.6721)
tensor(0.6704)
tensor(0.6693)
tensor(0.6684)


In [15]:
model(data)


tensor([[0.0270],
        [0.6540],
        [0.6502],
        [0.6540]], grad_fn=<AddmmBackward>)

TRAIN THE MODEL USING ENCRYPTED DEEP LEARNING TECHNIQUES

In [0]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)


In [17]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:11439637158 -> alice:61414560862]
 	-> (Wrapper)>[PointerTensor | me:38534742129 -> bob:50340357053]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:71029037493 -> alice:26451191762]
 	-> (Wrapper)>[PointerTensor | me:53555943619 -> bob:73263418076]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:45147454142 -> alice:41129993395]
 	-> (Wrapper)>[PointerTensor | me:34209330419 -> bob:18424697191]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:64107341283 -> alice:73070909700]
 	-> (Wrapper)>[PointerTensor | me:59504182076 -> bob:77259228

In [0]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)


In [19]:
encrypted_data


(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:67622010977 -> alice:25614320413]
	-> (Wrapper)>[PointerTensor | me:97792639575 -> bob:52509613543]
	*crypto provider: secure_worker*

In [0]:
encrypted_prediction = encrypted_model(encrypted_data)


In [21]:
encrypted_prediction.get().float_precision()


tensor([[0.0270],
        [0.6530],
        [0.6490],
        [0.6530]])

So , we see that we get same output in this case also.